In [1]:
import pickle
import cv2
import torch
from tqdm import tqdm

/home/sicong/.conda/envs/apex/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
import cv2
import numpy as np
from collections import defaultdict

def render_image_(example,predicted_boxes,all_gt_rooms,gt_boxes,output_dir):
    room_idx = {'living room':0,'living room 1':0,'living room 2':0, 'master room':1, 'kitchen':2, 'bathroom 1':3, 'bathroom 2':3, 'bathroom 3':3, 'dining room':4, 'common room 2':5, 'common room 3':5, 'common room 1':5, 'common room 4':5, 'balcony 1':9
            ,'balcony 2':9,'balcony 3':9, 'entrance':10, 'storage':11, 'bathroom': 3, 'balcony': 9, 'common room': 5, 'master room 2': 1, 'master room 1': 1, 'kitchen 1': 2, 'kitchen 2': 2,'master room 3': 1, 'storage room':11}

    color_idx = {15:(255,165,0),16:(255,165,0),17:(240,128,128),18:(240,128,128),0:(170,232,238),1:(0,165,255),5:(0,215,255),6:(255,215,0),7:(255,215,0),8:(255,215,0),2:(128,128,240),3:(230,216,173),4:(218,112,214),9:(35,142,107),10:(255,255,0),11:(221,160,221),12:(173,216,230),13:(107,142,35),14:(255,215,0),19:(255,165,0)}     

    image_height = 256
    image_width = 256
    number_of_color_channels = 3
    background_color = (255,255,255)
    gt_image = np.full((image_height, image_width, number_of_color_channels), background_color, dtype=np.uint8)
    predicted_image = np.full((image_height, image_width, number_of_color_channels), background_color, dtype=np.uint8)
    boundary_color = [0,0,0]


    left_boundary = int(example.boundary_tokens[0]['x_min'])

    
    living = defaultdict()
    common = defaultdict()
    master = defaultdict()
    balcony = defaultdict()
    bathroom = defaultdict()
    kitchen = defaultdict()
    storage = defaultdict()
    dining = defaultdict()
    for room in predicted_boxes:
        if room.startswith('living'):
            living[room] = predicted_boxes[room]
        elif room.startswith('common'):
            common[room] = predicted_boxes[room]
        elif room.startswith('master'):
            master[room] = predicted_boxes[room]
        elif room.startswith('balcony'):
            balcony[room] = predicted_boxes[room]
        elif room.startswith('bathroom'):
            bathroom[room] = predicted_boxes[room]
        elif room.startswith('kitchen'):
            kitchen[room] = predicted_boxes[room]
        elif room.startswith('storage'):
            storage[room] = predicted_boxes[room]
        elif room.startswith('dining'):
            dining[room] = predicted_boxes[room]
    room_type_list = [living,common,master,balcony,bathroom,kitchen,storage,dining]
    for room_type in room_type_list:
        for room in room_type:
            left_top_pr = (room_type[room][0],room_type[room][1])
            right_bt_pr = (room_type[room][2],room_type[room][3])

            color = color_idx[room_idx[room]] 
            # draw room on predicted image
            cv2.rectangle(predicted_image, left_top_pr, right_bt_pr, color,-1)

    for boundary_pixel in example.boundary:
        predicted_image[boundary_pixel[0],boundary_pixel[1]] = boundary_color
    
    
    # cv2.putText(predicted_image,"human",(10,10),0,0.3,boundary_color)
    cv2.imwrite(f'./{output_dir}/{example.image_id}.png', predicted_image)

In [56]:
with (open("/data/sicong/fp_data/dataset/Text2Design/pickle/Text2Design_artificial_all.pkl", "rb")) as openfile:
    data = pickle.load(openfile)

In [4]:
room_idx = {0:'living room', 1:'master room', 2:'kitchen', 3:'bathroom', 4:'dining room', 5:'common room', 6:'balcony'
            , 7:'entrance', 8:'storage'}
idx_room = {v: k for k, v in room_idx.items()}

In [50]:
from glob import glob

f = []
sub_dirs = glob("./data/T2D/gen_masks_2022_12_12_21_18_15/*/", recursive = True)
for i in range(len(sub_dirs)):
    f.append(sub_dirs[i].split('/')[-2])
len(f)


2308

In [29]:
# from os import walk

# f = []
# for (dirpath, dirnames, filenames) in walk('./data/T2D/gen_masks_2022_11_11_17_23_49/'):
#     f.extend(dirnames)
# len(f)

In [57]:
from collections import defaultdict
all_data_dict = defaultdict()
for i in tqdm(range(len(data))):
    sample = data[i]
    all_data_dict[sample['image_id'].split('.')[0]] = sample['rooms']

100%|██████████| 80788/80788 [00:00<00:00, 329448.94it/s]


In [53]:
eval_image_id = []
for eval_file in f:
    eval_image_id.append(eval_file.split('id')[-1])
len(eval_image_id)

2308

In [58]:
import os
import re

# if the string ends in digits m will be a Match object, or None otherwise.

# generate gt labels
for id in eval_image_id:
    fp = all_data_dict[id]
    if not os.path.exists('gt_boxes/image_id%s'%(id)):
        os.makedirs('gt_boxes/image_id%s'%(id))
    fout_filename = open('gt_boxes/image_id%s/boxes.txt'%(id), 'w')
    for i in range(len(fp)):
        # print(fp[i])
        room_type = fp[i]['room_type']
        m = re.search(r'\d+$', room_type)
        if m is not None:
            room_type = " ".join(room_type.split(" ")[:-1])
        r_idx = idx_room[room_type]
        # xmin ymin w h
        x = fp[i]['x']
        y = fp[i]['y']
        w = fp[i]['w']
        h = fp[i]['h']
        xmin = x - w/2.0
        ymin = y - h/2.0
        fout_filename.write('%.2f,%.2f,%.2f,%.2f,%s,0\n'%(xmin, ymin, w, h, r_idx))

In [13]:
# calculate IoU id0 as example
# load ground truth boxes
with open('./gt_boxes/image_id3908/boxes.txt') as f:
    lines = f.readlines()
    gt_boxes = defaultdict()
    for line in lines:
        res = line.split(",")[:-1]
        xmin = float(res[0])
        ymin = float(res[1])
        w = float(res[2])
        h = float(res[3])
        r_type = str(res[4])
        # gt_box = [  
        #     [int(xmin),int(ymin)],[int(xmin+w),int(ymin)],
        #     [int(xmin),int(ymin+h)],[int(xmin+w),int(ymin+h)]
        # ]
        gt_boxes[r_type] = (int(ymin), int(xmin), int(ymin+h), int(xmin+w))
f.close()
# load predicted boxes
with open('./data/T2D/gen_masks_2022_12_12_21_18_15/image_id3908/0/boxes.txt') as f:
    lines = f.readlines()
    pred_boxes = defaultdict()
    for line in lines:
        res = line.split(",")[:-1]
        xmin = float(res[0])
        ymin = float(res[1])
        w = float(res[2])
        h = float(res[3])
        r_type = str(res[4])
        # gt_box = [  
        #     [int(xmin),int(ymin)],[int(xmin+w),int(ymin)],
        #     [int(xmin),int(ymin+h)],[int(xmin+w),int(ymin+h)]
        # ]
        pred_boxes[r_type] = (int(ymin), int(xmin), int(ymin+h), int(xmin+w))
f.close()

In [27]:
# render image
# load ground truth boxes
room_idx = {0:'living room', 1:'master room', 2:'kitchen', 3:'bathroom', 4:'dining room', 5:'common room', 6:'balcony'
            , 7:'entrance', 8:'storage'}

with open('./gt_boxes/image_id3908/boxes.txt') as f:
    lines = f.readlines()
    gt_boxes = defaultdict()
    for line in lines:
        res = line.split(",")[:-1]
        xmin = int(float(res[0]))
        ymin = int(float(res[1]))
        ymax = int(ymin+float(res[2]))
        xmax = int(xmin+float(res[3]))
        room = room_idx[int(res[4])]
        # gt_box = [  
        #     [int(xmin),int(ymin)],[int(xmin+w),int(ymin)],
        #     [int(xmin),int(ymin+h)],[int(xmin+w),int(ymin+h)]
        # ]
        gt_boxes[room] = (int(ymin), int(xmin), int(ymax), int(xmax))
        if room in gt_boxes:
            if room+' 1' not in gt_boxes:
                gt_boxes[room+' 1'] = (ymin, xmin, ymax, xmax)
            else:
                if room+' 2' not in gt_boxes:
                    gt_boxes[room+' 2'] = (ymin, xmin, ymax, xmax)
                else:
                    if room+' 3' not in gt_boxes:
                        gt_boxes[room+' 3'] = (ymin, xmin, ymax, xmax)
                    else:
                        if room+' 4' not in gt_boxes:
                            gt_boxes[room+' 4'] = (ymin, xmin, ymax, xmax)
        else:
            gt_boxes[room] = (ymin, xmin, ymax, xmax)
f.close()
# load predicted boxes
with open('./data/T2D/gen_masks_2022_12_12_21_18_15/image_id3908/0/boxes.txt') as f:
    lines = f.readlines()
    pred_boxes = defaultdict()
    for line in lines:
        res = line.split(",")[:-1]
        xmin = int(float(res[0]))
        ymin = int(float(res[1]))
        ymax = int(ymin+float(res[2]))
        xmax = int(xmin+float(res[3]))
        room = room_idx[int(res[4])]
        # gt_box = [  
        #     [int(xmin),int(ymin)],[int(xmin+w),int(ymin)],
        #     [int(xmin),int(ymin+h)],[int(xmin+w),int(ymin+h)]
        # ]
        pred_boxes[room] = (int(ymin), int(xmin), int(ymax), int(xmax))
        if room in pred_boxes:
            if room+' 1' not in pred_boxes:
                pred_boxes[room+' 1'] = (ymin, xmin, ymax, xmax)
            else:
                if room+' 2' not in pred_boxes:
                    pred_boxes[room+' 2'] = (ymin, xmin, ymax, xmax)
                else:
                    if room+' 3' not in pred_boxes:
                        pred_boxes[room+' 3'] = (ymin, xmin, ymax, xmax)
                    else:
                        if room+' 4' not in pred_boxes:
                            pred_boxes[room+' 4'] = (ymin, xmin, ymax, xmax)
        else:
            pred_boxes[room] = (ymin, xmin, ymax, xmax)
f.close()

In [31]:
import cv2
import numpy as np
room_idx = {'living room':0,'living room 1':0,'living room 2':0, 'master room':1, 'kitchen':2, 'bathroom 1':3, 'bathroom 2':3, 'bathroom 3':3, 'dining room':4, 'common room 2':5, 'common room 3':5, 'common room 1':5, 'common room 4':5, 'balcony 1':9
            ,'balcony 2':9,'balcony 3':9, 'entrance':10, 'storage':11, 'bathroom': 3, 'balcony': 9, 'common room': 5, 'master room 2': 1, 'master room 1': 1, 'kitchen 1': 2, 'kitchen 2': 2,'master room 3': 1, 'storage room':11}

color_idx = {15:(255,165,0),16:(255,165,0),17:(240,128,128),18:(240,128,128),0:(170,232,238),1:(0,165,255),5:(0,215,255),6:(255,215,0),7:(255,215,0),8:(255,215,0),2:(128,128,240),3:(230,216,173),4:(218,112,214),9:(35,142,107),10:(255,255,0),11:(221,160,221),12:(173,216,230),13:(107,142,35),14:(255,215,0),19:(255,165,0)}     

def render_image(example,predicted_boxes,all_gt_rooms,gt_boxes,output_dir):
    image_height = 256
    image_width = 256
    number_of_color_channels = 3
    background_color = (255,255,255)
    gt_image = np.full((image_height, image_width, number_of_color_channels), background_color, dtype=np.uint8)
    predicted_image = np.full((image_height, image_width, number_of_color_channels), background_color, dtype=np.uint8)
    boundary_color = [0,0,0]


    left_boundary = int(example.boundary_tokens[0]['x_min'])
    # # draw compass
    # start_point = [240, 40]
    # end_point = [240, 20]
    # thickness = 2
    # cv2.arrowedLine(predicted_image, start_point, end_point,
    #                             boundary_color, thickness,tipLength=0.35)
    # cv2.putText(predicted_image,"N",(230,35),0,0.3,(0,0,0))

    # # draw measure
    # start_point = [left_boundary, 245]
    # end_point = [left_boundary+40, 245]
    # cv2.line(predicted_image, start_point, end_point, boundary_color, thickness)

    # start_point = [left_boundary, 245]
    # end_point = [left_boundary, 240]
    # cv2.line(predicted_image, start_point, end_point, boundary_color, thickness)

    # start_point = [left_boundary+40, 245]
    # end_point = [left_boundary+40, 240]
    # cv2.line(predicted_image, start_point, end_point, boundary_color, thickness)
    # cv2.putText(predicted_image,"10feet",(left_boundary,235),0,0.3,(0,0,0))
    


    # draw boundary
    for boundary_pixel in example.boundary:
        gt_image[boundary_pixel[0],boundary_pixel[1]] = boundary_color
        predicted_image[boundary_pixel[0],boundary_pixel[1]] = boundary_color


    for room in predicted_boxes:
        if room in all_gt_rooms:
            left_top_pr = (predicted_boxes[room][0],predicted_boxes[room][1])
            right_bt_pr = (predicted_boxes[room][2],predicted_boxes[room][3])
            left_top_gt = (gt_boxes[room][0],gt_boxes[room][1])
            right_bt_gt = (gt_boxes[room][2],gt_boxes[room][3])

            color = color_idx[room_idx[room]]
            # draw room on ground truth image
            cv2.rectangle(gt_image, left_top_gt, right_bt_gt, color,2)
            #cv2.putText(gt_image,str(room),(left_top_gt[0]+5,left_top_gt[1]+10),0,0.3,color)
            # draw room on predicted image
            cv2.rectangle(predicted_image, left_top_pr, right_bt_pr, color,2)
            #cv2.putText(predicted_image,str(room),(left_top_pr[0]+5,left_top_pr[1]+10),0,0.3,color)

        else:
            left_top_pr = (predicted_boxes[room][0],predicted_boxes[room][1])
            right_bt_pr = (predicted_boxes[room][2],predicted_boxes[room][3])

            color = color = color_idx[room_idx[room]] 
            # draw room on predicted image
            cv2.rectangle(predicted_image, left_top_pr, right_bt_pr, color,2)
            #cv2.putText(predicted_image,str(room),(left_top_pr[0]+5,left_top_pr[1]+10),0,0.3,color)
    
    for room in all_gt_rooms:
        if room not in predicted_boxes:
            left_top_gt = (gt_boxes[room][0],gt_boxes[room][1])
            right_bt_gt = (gt_boxes[room][2],gt_boxes[room][3])
            color = color = color = color_idx[room_idx[room]]  # randomize a color for one specific room
            # draw room on ground truth image
            cv2.rectangle(gt_image, left_top_gt, right_bt_gt, color,2)
            #cv2.putText(gt_image,str(room),(left_top_gt[0]+5,left_top_gt[1]+10),0,0.3,color)
    
    cv2.putText(predicted_image,"human",(10,10),0,0.3,boundary_color)
    cv2.putText(gt_image,"ground truth",(10,10),0,0.3,boundary_color)

    im_h = cv2.hconcat([gt_image, predicted_image])
    # cv2.imshow('image',im_h)
    # cv2.waitKey()
    # cv2.destroyAllWindows()
    # cv2.imwrite(f'{output_dir}output_images/{example.image_id}.png', im_h)
    cv2.imwrite(f'./{example.image_id}.png', im_h)


In [14]:
from shapely.geometry import box
from shapely.ops import unary_union
def calculate_iou(gt_boxes, predicted_boxes):
    # group by same room type
    gt_group = defaultdict()
    for gt_b in gt_boxes.keys():
        if gt_b not in gt_group:
            gt_group[gt_b.split()[0]] = [gt_b]
        else:
            gt_group[gt_b.split()[0]].append(gt_b)
    pre_group = defaultdict()
    for gt_b in predicted_boxes.keys():
        if gt_b not in pre_group:
            pre_group[gt_b.split()[0]] = [gt_b]
        else:
            pre_group[gt_b.split()[0]].append(gt_b)
    # ymin, xmin, ymax, xmax = box

    # macro average IoU
    union = 0
    intersection = 0
    gt_group1 = gt_group.copy()
    for group in pre_group.keys():
        if group in gt_group1.keys():
            
            group1 = pre_group[group]
            g1_boxes = []
            for r1 in group1:
                g1_boxes.append(box( predicted_boxes[r1][0],predicted_boxes[r1][1],predicted_boxes[r1][2],predicted_boxes[r1][3] ))

            group2 = gt_group1[group]
            g2_boxes = []
            for r2 in group2:
                g2_boxes.append(box( gt_boxes[r2][0],gt_boxes[r2][1],gt_boxes[r2][2],gt_boxes[r2][3] ))
            
            all_boxes = g1_boxes + g2_boxes
            all_union = unary_union(all_boxes)
            all_union_area = all_union.area
            union += all_union_area

            union1 = unary_union(g1_boxes)
            union2 = unary_union(g2_boxes)
            all_intersection = union1.intersection(union2)
            all_intersection_area = all_intersection.area
            intersection += all_intersection_area
            gt_group1.pop(group)
        else: # if predicted doesnt exist in ground truth
            group1 = pre_group[group]
            g1_boxes = []
            for r1 in group1:
                g1_boxes.append(box( predicted_boxes[r1][0],predicted_boxes[r1][1],predicted_boxes[r1][2],predicted_boxes[r1][3] ))
            all_union = unary_union(g1_boxes)
            all_union_area = all_union.area
            all_intersection_area = 0
            union += all_union_area
    for group in gt_group1.keys(): # if ground truth doesnt exist in predicted
        group2 = gt_group1[group]
        g2_boxes = []
        for r2 in group2:
            g2_boxes.append(box( gt_boxes[r2][0],gt_boxes[r2][1],gt_boxes[r2][2],gt_boxes[r2][3] ))
        all_union = unary_union(g2_boxes)
        all_union_area = all_union.area
        all_intersection_area = 0
        union += all_union_area
    if union != 0:
        macro_average_iou = intersection/union
    else:
        macro_average_iou = 0
    
    # micro average IoU
    num = 0
    total_iou = 0
    gt_group2 = gt_group.copy()
    for group in pre_group.keys():
        if group in gt_group2.keys():
            
            group1 = pre_group[group]
            g1_boxes = []
            for r1 in group1:
                g1_boxes.append(box( predicted_boxes[r1][0],predicted_boxes[r1][1],predicted_boxes[r1][2],predicted_boxes[r1][3] ))

            group2 = gt_group2[group]
            g2_boxes = []
            for r2 in group2:
                g2_boxes.append(box( gt_boxes[r2][0],gt_boxes[r2][1],gt_boxes[r2][2],gt_boxes[r2][3] ))
            
            all_boxes = g1_boxes + g2_boxes
            all_union = unary_union(all_boxes)
            all_union_area = all_union.area
            #union += all_union_area

            union1 = unary_union(g1_boxes)
            union2 = unary_union(g2_boxes)
            all_intersection = union1.intersection(union2)
            all_intersection_area = all_intersection.area
            #intersection += all_intersection_area
            total_iou += all_intersection_area/all_union_area
            num += 1
            gt_group2.pop(group)
        else: # if predicted doesnt exist in ground truth
            num += 1
    for group in gt_group2.keys(): # if ground truth doesnt exist in predicted
        num += 1
    if num != 0:
        micro_average_iou = total_iou/num
    else:
        micro_average_iou = 0
    
    return macro_average_iou, micro_average_iou

In [15]:
macro_average_iou, micro_average_iou = calculate_iou(gt_boxes, pred_boxes)

In [16]:
print(macro_average_iou)

0.22276839780579896


In [18]:
import json
  
f = open('floorplan_eval.json')

data = json.load(f)

In [10]:
class Example:
    def __init__(self, image_id, boundary,boundary_tokens, rooms):
        self.image_id = image_id
        self.boundary = boundary
        self.rooms = rooms
        self.boundary_tokens = boundary_tokens

In [20]:
for d in data:
    if d['img_id'] == '3908':
        boundary = d['boundary']
        rooms = d['rooms']
        boundary_tokens = d['boundary_boxs']
        example = Example('3908', boundary,boundary_tokens, rooms)

In [25]:
print(gt_boxes)

defaultdict(None, {'5': (55, 70, 99, 114), '1': (43, 150, 110, 194), '6': (140, 40, 154, 75), '2': (92, 81, 136, 102), '3': (51, 121, 82, 146), '0': (66, 89, 202, 187)})


In [34]:
render_image(example, pred_boxes,gt_boxes,gt_boxes,'./')

In [12]:
len(eval_image_id)

2308

In [6]:
from glob import glob
f = []
sub_dirs = glob("./data/T2D/gen_masks_2022_12_12_21_18_15/*/", recursive = True)
for i in range(len(sub_dirs)):
    f.append(sub_dirs[i].split('/')[-2])
len(f)
eval_image_id = []

for eval_file in f:
    eval_image_id.append(eval_file.split('id')[-1])
len(eval_image_id)
from collections import defaultdict

In [14]:
# generate gt labels
for id in eval_image_id:
    fp = all_data_dict[id]
    if not os.path.exists('gt_boxes/image_id%s'%(id)):
        os.makedirs('gt_boxes/image_id%s'%(id))
    fout_filename = open('gt_boxes/image_id%s/boxes.txt'%(id), 'w')
    for i in range(len(fp)):
        # print(fp[i])
        room_type = fp[i]['room_type']
        m = re.search(r'\d+$', room_type)
        if m is not None:
            room_type = " ".join(room_type.split(" ")[:-1])
        r_idx = idx_room[room_type]
        # xmin ymin w h
        x = fp[i]['x']
        y = fp[i]['y']
        w = fp[i]['w']
        h = fp[i]['h']
        xmin = x - w/2.0
        ymin = y - h/2.0
        fout_filename.write('%.2f,%.2f,%.2f,%.2f,%s,0\n'%(xmin, ymin, w, h, r_idx))

NameError: name 'all_data_dict' is not defined

In [60]:
import json
  
f = open('floorplan_eval.json')

data_ = json.load(f)

In [33]:
room_idx = {0:'living room', 1:'master room', 2:'kitchen', 3:'bathroom', 4:'dining room', 5:'common room', 6:'balcony'
            , 7:'entrance', 8:'storage'}

In [68]:
def cal_iou_all_(predicted_path):
    sum_macro_iou = 0
    sum_micro_iou = 0
    counts = 0
    for img_id in eval_image_id:
        # load ground truth boxes
        with open(f'./gt_boxes/image_id{img_id}/boxes.txt') as f:
            lines = f.readlines()
            gt_boxes = defaultdict()
            for line in lines:
                res = line.split(",")[:-1]
                xmin = int(float(res[0]))
                ymin = int(float(res[1]))
                ymax = int(ymin+float(res[2]))
                xmax = int(xmin+float(res[3]))
                room = room_idx[int(res[4])]
                # gt_box = [  
                #     [int(xmin),int(ymin)],[int(xmin+w),int(ymin)],
                #     [int(xmin),int(ymin+h)],[int(xmin+w),int(ymin+h)]
                # ]
                gt_boxes[room] = (int(ymin), int(xmin), int(ymax), int(xmax))
                if room in gt_boxes:
                    if room+' 1' not in gt_boxes:
                        gt_boxes[room+' 1'] = (ymin, xmin, ymax, xmax)
                    else:
                        if room+' 2' not in gt_boxes:
                            gt_boxes[room+' 2'] = (ymin, xmin, ymax, xmax)
                        else:
                            if room+' 3' not in gt_boxes:
                                gt_boxes[room+' 3'] = (ymin, xmin, ymax, xmax)
                            else:
                                if room+' 4' not in gt_boxes:
                                    gt_boxes[room+' 4'] = (ymin, xmin, ymax, xmax)
                else:
                    gt_boxes[room] = (ymin, xmin, ymax, xmax)
        f.close()
        # load predicted boxes, predicted_path = gen_masks_2022_11_11_17_23_49
        with open(f'./data/T2D/{predicted_path}/image_id{img_id}/0/boxes.txt') as f:
            lines = f.readlines()
            pred_boxes = defaultdict()
            for line in lines:
                res = line.split(",")[:-1]
                xmin = int(float(res[0]))
                ymin = int(float(res[1]))
                ymax = int(ymin+float(res[2]))
                xmax = int(xmin+float(res[3]))
                room = room_idx[int(res[4])]
                # gt_box = [  
                #     [int(xmin),int(ymin)],[int(xmin+w),int(ymin)],
                #     [int(xmin),int(ymin+h)],[int(xmin+w),int(ymin+h)]
                # ]
                pred_boxes[room] = (int(ymin), int(xmin), int(ymax), int(xmax))
                if room in pred_boxes:
                    if room+' 1' not in pred_boxes:
                        pred_boxes[room+' 1'] = (ymin, xmin, ymax, xmax)
                    else:
                        if room+' 2' not in pred_boxes:
                            pred_boxes[room+' 2'] = (ymin, xmin, ymax, xmax)
                        else:
                            if room+' 3' not in pred_boxes:
                                pred_boxes[room+' 3'] = (ymin, xmin, ymax, xmax)
                            else:
                                if room+' 4' not in pred_boxes:
                                    pred_boxes[room+' 4'] = (ymin, xmin, ymax, xmax)
                else:
                    pred_boxes[room] = (ymin, xmin, ymax, xmax)
        f.close()
        for d in data_:
            if d['img_id'].split('.')[0] == img_id:
                boundary = d['boundary']
                rooms = d['rooms']
                boundary_tokens = d['boundary_boxs']
                example = Example(img_id, boundary,boundary_tokens, rooms)
        render_image_(example,pred_boxes,gt_boxes,gt_boxes,'draw_arti_only')
    return
    #     macro_average_iou, micro_average_iou = calculate_iou(gt_boxes, pred_boxes)
    #     sum_macro_iou += macro_average_iou
    #     sum_micro_iou += micro_average_iou
    #     counts += 1
    # avg_macro = sum_macro_iou/counts
    # avg_micro = sum_micro_iou/counts
    # return avg_macro, avg_micro

In [69]:
cal_iou_all_('gen_masks_2022_12_01_17_05_59')

In [5]:
cal_iou_all('gen_masks_2022_12_06_03_02_59')

(0.19299291610378763, 0.10903618736042409)